In [14]:
import openpyxl
import numpy as np
import time
import io
import requests
import pandas as pd
import ipywidgets as widgets
from tqdm import tqdm
from pytrends.request import TrendReq
from IPython.display import display

# Analiza Google Trends

Prześlij plik w formacie **.xlsx** z tytułami gier, a następnie kliknij "wczytaj tytuły"
Pamiętaj, żeby każdy tytuł gry znajdował się w osobnej komórce. Tak powinna wyglądać tabela:


<img src="https://i.ibb.co/GVRCXF9/Zrzut-ekranu-2022-06-20-o-11-17-34.png" title="Przykładowa tabela"/>

In [15]:
EXACT_KEYWORDS = []
TITLES_LIST = []
pytrend = TrendReq()

In [16]:
uploader = widgets.FileUpload(
    accept='.xlsx',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
display(uploader)

FileUpload(value={}, accept='.xlsx', description='Upload')

In [17]:
def validate(uploaded_file):

    global EXACT_KEYWORDS
    global TITLES_LIST
    try:
        input_file = list(uploaded_file.value.values())[0]
        content = input_file['content']
        df_upload = pd.read_excel(content, header=None)
        df_upload = df_upload.replace(r'\n','', regex=True).replace(';','', regex=True)
        keywords = df_upload.iloc[:, 0].to_list()
        keywords = [k for k in keywords if k]

        keywords_codes = []
        for keyword in tqdm(keywords):
            data = (pytrend.suggestions(keyword))
            dicti = next(
                (item for item in data if "game" in item["type"]),
                {
                    'mid': keyword,
                    'title': keyword,
                    'type': "0"
                }
            )
            if dicti is not None:
                keywords_codes.append(dicti)
            else:
                continue
                
        df_codes = pd.DataFrame(keywords_codes)

        global WRONG_ID_DF
        WRONG_ID_DF = df_codes.loc[~df_codes["type"].str.contains("game")].drop(columns='mid')
        WRONG_ID_DF['type'].apply(pd.to_numeric)
        WRONG_ID_DF['type'] = "0.00"
        WRONG_ID_DF.rename(
            columns={
                'type': 'średnia',
                'title': 'tytuł'
                },
            inplace=True
        )
        
        WRONG_ID_DF = WRONG_ID_DF.set_index('tytuł')
        
        final_df = df_codes.loc[df_codes["type"].str.contains("game")]

        EXACT_KEYWORDS = final_df['mid'].to_list()
        TITLES_LIST = final_df['title'].to_list()
        print("Tytuły wczytane poprawnie")
        

    except IndexError:
        print("Błąd: nie przesłano pliku bądź przesłano pusty plik")
    except requests.exceptions.Timeout:
        print("Błąd: Google nie odpowiada. Spróbuj ponownie później")
    except:
        print("Coś poszło nie tak, spróbuj ponownie później")

    
    

In [18]:
button2 = widgets.Button(description="wczytaj tytuły")
output2 = widgets.Output()

display(button2, output2)


@output2.capture(clear_output=True)
def on_button_clicked_validate(b):
    with output2:
        validate(uploader)


button2.on_click(on_button_clicked_validate)

Button(description='wczytaj tytuły', style=ButtonStyle())

Output()

In [19]:
time_frame = widgets.Dropdown(
    options=[
        ("ostatni miesiąc", "today 1-m"),
        ("ostatnie 3 miesiące", "today 3-m"),
        ("ostatni rok", "today 12-m"),
        ("ostatnie 5 lat", "today 5-y"),
        ("od 2004-", "all"),
        ("własna data", "custom_date")
    ],
    value='today 12-m',
    description='Zakres czasu:',
    disabled=False,
)

In [20]:
custom_timeframe_start = widgets.DatePicker(
    description='Początek',
    disabled=False
)
custom_timeframe_stop = widgets.DatePicker(
    description='Koniec',
    disabled=False
)

In [21]:
location = widgets.Dropdown(
    options=[
        ('cały świat', ''),
        ('Stany Zjednoczone', 'US'),
        ('Polska', 'PL'),
        ('Wielka Brytania', 'GB'),
        ('Niemcy', 'DE'),
        ('Japonia', 'JP'),
        ('Rosja', 'RU'),
        ('Korea Południowa', 'KR')
    ], 
    value='',
    description='Region:',
    disabled=False,
)

Określ parametry, dla których chcesz ściągnąć dane - zakres czasu oraz region.

Jeśli chcesz określić własny przedział czasu, w pierwszej komórce wybierz "własna data", a następnie wypełnij datę początkową i końcową. 

Jeśli chcesz ściągnąć dane dla zdefiniowanego zakresu (np. ostatni rok) **wystarczy, że zaznaczysz to w pierwszej komórce.**

Kliknij "wygeneruj" by uzyskać raport. **Zawartość tabeli kopiuje się automatycznie.** Wystarczy wkleić do Excela.

In [22]:
display(custom_timeframe_start, custom_timeframe_stop)

DatePicker(value=None, description='Początek')

DatePicker(value=None, description='Koniec')

In [23]:

display(time_frame)

display(location)

Dropdown(description='Zakres czasu:', index=2, options=(('ostatni miesiąc', 'today 1-m'), ('ostatnie 3 miesiąc…

Dropdown(description='Region:', options=(('cały świat', ''), ('Stany Zjednoczone', 'US'), ('Polska', 'PL'), ('…

In [2]:
def compare(kw_list):

    trends = dict()
    try:
        for i in tqdm(kw_list):
            time.sleep(1)
            if time_frame.value == "custom_date":
                zakres = str(custom_timeframe_start.value) + " " + str(custom_timeframe_stop.value)
                pytrend.build_payload(
                    [i, '/m/0c7kdf'],
                    cat=0,
                    timeframe=zakres,
                    geo=location.value
                )
            else:
                zakres = time_frame.value
                pytrend.build_payload(
                    [i, '/m/0c7kdf'],
                    cat=0,
                    timeframe=zakres,
                    geo=location.value
                )
            trends[i] = pytrend.interest_over_time()[i]

        trends_df = pd.DataFrame.from_dict(trends)
        trends_df.columns = TITLES_LIST

        av_column = trends_df.mean(axis=0, numeric_only=True).round(2).sort_values(ascending=False)
        av_column = av_column.to_frame(name="średnia")
        
        final_av_column = pd.concat([av_column, WRONG_ID_DF])
        
        display(final_av_column)
        
        if time_frame.value == "custom_date":
            print(("zakres: " + zakres + ';'), ("region: " + location.label))
        else:
            print(("zakres: " + time_frame.label + ';'), ("region: " + location.label))

        final_av_column["średnia"] = final_av_column["średnia"].map(str)
        final_av_column["średnia"] = final_av_column["średnia"].str.replace(".", ",", regex=True)
        final_av_column.to_clipboard()

    except requests.exceptions.Timeout:
        print("Błąd: Google nie odpowiada. Spróbuj ponownie później")
    except NameError:
        print("Błąd: nie przesłano pliku")
    except PyperclipException:
        print("Automatyczne kopiowanie tymczasowo zablokowane")
    except:
        print("Coś poszło nie tak")

In [3]:
button = widgets.Button(description="wygeneruj")
output = widgets.Output()

display(button, output)


@output.capture(clear_output=True)
def on_button_clicked(b):
    with output:
        compare(EXACT_KEYWORDS)


button.on_click(on_button_clicked)

NameError: name 'widgets' is not defined